In [20]:
import os
os.getcwd()

'/home/onyxia/work/hackathon-un-2022'

In [21]:
os.chdir("..")
import utils.functions as fc

In [28]:
from importlib import reload
fc = reload(fc)

In [22]:
ship_data_enriched = fc.create_ship_data_enriched()

In [23]:
ship_data_enriched.columns

Index(['LRIMOShipNo', 'StatCode5', 'AlterationsDescriptiveNarrative',
       'PropulsionTypeCode', 'ShipName', 'ExName',
       'MaritimeMobileServiceIdentityMMSINumber', 'RegisteredOwnerCode',
       'RegisteredOwnerCountryOfRegistration',
       'RegisteredOwnerCountryofDomicile',
       ...
       'ShipTypeLevel4', 'Level3Code', 'ShipTypeLevel3', 'Level2Code',
       'ShipTypeLevel2', 'ShipTypeLevel1Code', 'ShiptypeLevel1', 'HullType',
       'SubGroup', 'SubType'],
      dtype='object', length=119)

In [29]:
AIS = fc.read_ais_parquet()

In [30]:
AIS.shape

(1543295, 9)

In [26]:
ship_data_enriched = ship_data_enriched.loc[ship_data_enriched["ShiptypeLevel1"] == "Cargo Carrying",:]

In [27]:
AIS.shape

(179988, 8)

In [10]:
ship_data_enriched = ship_data_enriched.loc[ship_data_enriched["ShipTypeLevel3"].isin(["General Cargo",
                                                                                      "Bulk Dry", 
                                                                                      "Container", 
                                                                                      "Other Bulk Dry",
                                                                                      "Refrigerated Cargo",
                                                                                      "Other Dry Cargo",
                                                                                      "Bulk Dry / Oil",
                                                                                      "Self Discharging Bulk Dry"]),:]

In [11]:
ship_data_enriched["ShipTypeLevel4"].value_counts()

General Cargo Ship               39960
Bulk Carrier                     20379
Container Ship                    9275
Refrigerated Cargo Ship           2141
Aggregates Carrier                1204
Cement Carrier                     818
Ore Carrier                        810
Deck Cargo Ship                    717
Livestock Carrier                  388
Wood Chips Carrier                 321
Heavy Load Carrier                 320
Self Discharging Bulk Carrier      314
Ore/Oil Carrier                    252
Bulk/Oil Carrier                   215
Limestone Carrier                   75
Palletised Cargo Ship               70
Barge Carrier                       39
Nuclear Fuel Carrier                23
Powder Carrier                       8
Urea Carrier                         8
Passenger/Container Ship             5
Pulp Carrier                         3
Refined Sugar Carrier                2
Name: ShipTypeLevel4, dtype: int64

In [12]:
AIS.shape # 179 988

(179988, 8)

In [15]:
AIS.loc[AIS["mmsi"].isna(), :].shape

(0, 8)

In [16]:
AIS_enriched = AIS.merge(
    ship_data_enriched, left_on = 'mmsi',
    right_on = "MaritimeMobileServiceIdentityMMSINumber", how="left")

In [17]:
AIS_enriched.head()

,hex_resolution,longitude,mmsi,eeid,H3_int_index_8,latitude,dt_insert_utc,polygon_name,LRIMOShipNo,StatCode5,...,ShipTypeLevel4,Level3Code,ShipTypeLevel3,Level2Code,ShipTypeLevel2,ShipTypeLevel1Code,ShiptypeLevel1,HullType,SubGroup,SubType
0,8,33.333333,272168100,5281533799698068644,613022282759012351,46.758333,2022-01-01 23:53:28,Polygon,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8,32.910907,272686000,4861284512643996592,613022403708059647,46.107775,2022-01-01 21:25:23,Polygon,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8,32.910883,272686000,4861284512643996592,613022403708059647,46.107767,2022-01-01 11:31:20,Polygon,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8,32.910882,272686000,4861284512643996592,613022403708059647,46.107755,2022-01-01 12:40:23,Polygon,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8,32.910875,272686000,4861284512643996592,613022403708059647,46.107778,2022-01-01 14:40:21,Polygon,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
AIS_enriched.shape

(179988, 127)

In [19]:
AIS_enriched.columns

Index(['hex_resolution', 'longitude', 'mmsi', 'eeid', 'H3_int_index_8',
       'latitude', 'dt_insert_utc', 'polygon_name', 'LRIMOShipNo', 'StatCode5',
       ...
       'ShipTypeLevel4', 'Level3Code', 'ShipTypeLevel3', 'Level2Code',
       'ShipTypeLevel2', 'ShipTypeLevel1Code', 'ShiptypeLevel1', 'HullType',
       'SubGroup', 'SubType'],
      dtype='object', length=127)

In [20]:
mmsi_number = pd.read_csv(fs.open(f'{bucket}/AIS/mmid.csv',
                         mode='rb'), error_bad_lines=False, encoding='iso-8859-1', sep=";")

/tmp/ipykernel_930/1136292346.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  mmsi_number = pd.read_csv(fs.open(f'{bucket}/AIS/mmid.csv',


In [21]:
mmsi_number.head()

,Digit,Allocated to
0,201,Albania (Republic of)
1,202,Andorra (Principality of)
2,203,Austria
3,204,Portugal - Azores
4,205,Belgium


In [25]:
AIS_enriched["Digit"] = AIS_enriched["mmsi"].astype(str).str[0:3]

In [24]:
AIS_enriched["mmsi_digits"].head()

0    272
1    272
2    272
3    272
4    272
Name: mmsi_digits, dtype: object

In [29]:
mmsi_number["Digit"] = mmsi_number["Digit"].astype(str)

In [30]:
AIS_enriched2 = pd.merge(AIS_enriched, mmsi_number, on=["Digit"], how="left")

In [31]:
AIS_enriched2.head()

,hex_resolution,longitude,mmsi,destination,eeid,H3_int_index_8,latitude,dt_insert_utc,polygon_name,StatCode5,...,ShipTypeLevel1,ShipTypeLevel2,ShipTypeLevel3,ShipTypeLevel4,ShipTypeLevel5,SubGroup,SubType,mmsi_digits,Digit,Allocated to
0,8,31.408333,272157700,MYKOLAIV,5191743282127358980,613021963599740927,47.526667,2022-01-01 05:58:49,Polygon,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,272,272,Ukraine
1,8,31.331667,272157700,MYKOLAIV,5191743282127358980,613021966030340095,47.535000,2022-01-01 18:25:52,Polygon,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,272,272,Ukraine
2,8,31.331667,272157700,MYKOLAIV,5191743282127358980,613021966030340095,47.535000,2022-01-01 21:14:30,Polygon,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,272,272,Ukraine
3,8,31.333333,272157700,MYKOLAIV,5191743282127358980,613021966030340095,47.535000,2022-01-01 07:46:57,Polygon,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,272,272,Ukraine
4,8,31.333333,272157700,MYKOLAIV,5191743282127358980,613021966030340095,47.535000,2022-01-01 09:43:50,Polygon,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,272,272,Ukraine


In [37]:
AIS_enriched2.loc[AIS_enriched2["destination"].isna(), :].shape[0]/AIS_enriched2.shape[0] # 377 313, 24 %

0.24448533818874552

In [34]:
AIS_enriched2.loc[~AIS_enriched2["destination"].isna(), :].shape # 1 165 982

(1165982, 27)

In [40]:
AIS_enriched2 = AIS_enriched2.rename({"Allocated to" : "origin_country"}, axis=1)

In [41]:
AIS_enriched2["origin_country"].value_counts()

Russian Federation                                                               296823
Ukraine                                                                          233616
Romania                                                                          205072
Republic of Türkiye                                                              138906
Panama (Republic of)                                                              96314
                                                                                  ...  
Syrian Arab Republic                                                                 90
Seychelles (Republic of)                                                             76
Belgium                                                                               4
Japan                                                                                 1
United Kingdom of Great Britain and Northern Ireland - British Virgin Islands         1
Name: origin_country, Length: 71

In [44]:
AIS_enriched2.loc[AIS_enriched2["origin_country"]=="",:].shape

(0, 27)

In [45]:
AIS_enriched2.loc[AIS_enriched2["origin_country"].isna(), :].shape

(0, 27)

In [42]:
# a present on alloue à chaque port son pays
import pandas as pd
import s3fs
import os
s3_url = "https://" + os.environ["AWS_S3_ENDPOINT"]
conn = s3fs.S3FileSystem(client_kwargs={'endpoint_url': s3_url})
bucket = "projet-hackathon-un-2022/open-data/wfp_ports/"

In [ ]:
path2data = bucket + ".csv"
with conn.open(path2data, mode="rb") as file_in:
     ports = pd.read_csv(file_in, error_bad_lines=False, encoding='iso-8859-1')